## Model evaluation & Text Geneation

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

vocab_size = 10000
sentence_len = 20
pred_len = 3
train_len = sentence_len - pred_len

# load tokenizer
with open('tokenizer.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [12]:
learning_rate = 0.02
m1_embedding_dim=50
m1_lstm_hidden = 100
m1_dense_features = 100
m1_dropout = 0.3

class Model1(nn.Module):
    
    def __init__(self):
        super(Model1, self).__init__()
        self.embed = nn.Embedding(  num_embeddings=vocab_size + 1, 
                                    embedding_dim=m1_embedding_dim, 
                                    padding_idx=train_len)
        self.rnn = nn.LSTM(input_size=m1_embedding_dim, 
                            hidden_size=m1_lstm_hidden, 
                            num_layers=2)
        self.dense1 = nn.Linear(in_features=m1_lstm_hidden, 
                                out_features=m1_dense_features)
        self.dense2 = nn.Linear(in_features=m1_dense_features, 
                                out_features=vocab_size)
    
    def forward(self, x):
        x = self.embed(x)
        x, _ = self.rnn(x)  # _ = (h, c)
        x = x[:,-pred_len:,:]  # get pred_len last steps of output
        x = F.relu( self.dense1(x) )
        x = F.dropout(x, p=m1_dropout)
        x = self.dense2(x)
        # x = F.log_softmax(x)  # this step is part of CrossEntropyLoss
        return x


model1 = Model1().to(device)
optimizer1 = optim.Adam(model1.parameters(), lr=learning_rate)

In [13]:
learning_rate = 0.02
m2_embedding_dim=50
m2_gru_hidden = 100
m2_dense_features = 100
m2_dropout = 0.3

class Model2(nn.Module):
    
    def __init__(self):
        super(Model2, self).__init__()
        self.embed = nn.Embedding(  num_embeddings=vocab_size + 1, 
                                    embedding_dim=m2_embedding_dim, 
                                    padding_idx=train_len)
        self.rnn = nn.GRU(input_size=m1_embedding_dim, 
                            hidden_size=m2_gru_hidden, 
                            num_layers=3)
        self.dense1 = nn.Linear(in_features=m2_gru_hidden, 
                                out_features=m2_dense_features)
        self.dense2 = nn.Linear(in_features=m2_dense_features, 
                                out_features=vocab_size)
    
    def forward(self, x):
        x = self.embed(x)
        x, _ = self.rnn(x)  # _ = (h, c)
        x = x[:,-pred_len:,:]  # get last steps of output
        x = F.relu( self.dense1(x) )
        x = F.dropout(x, p=m1_dropout)
        x = self.dense2(x)
        # x = F.log_softmax(x)  # this step is part of CrossEntropyLoss
        return x


model2 = Model2().to(device)
optimizer2 = optim.Adam(model2.parameters(), lr=learning_rate)

In [14]:
learning_rate = 0.06
m3_embedding_dim=50
m3_lstm_hidden = 40
m3_lstm_dropout = 0.1
m3_dense_features = 100
m3_dropout = 0.3

class Model3(nn.Module):
    
    def __init__(self):
        super(Model3, self).__init__()
        self.embed = nn.Embedding(  num_embeddings=vocab_size + 1, 
                                    embedding_dim=m3_embedding_dim, 
                                    padding_idx=train_len)
        self.rnn = nn.LSTM(input_size=m3_embedding_dim, 
                            hidden_size=m3_lstm_hidden, 
                            num_layers=3,
                            dropout=m3_lstm_dropout)
        self.dense1 = nn.Linear(in_features=m3_lstm_hidden, 
                                out_features=m3_dense_features)
        self.dense2 = nn.Linear(in_features=m3_dense_features, 
                                out_features=vocab_size)
    
    def forward(self, x):
        x = self.embed(x)
        x, _ = self.rnn(x)  # _ = (h, c)
        x = x[:,-pred_len:,:]  # get last steps of output
        x = F.relu( self.dense1(x) )
        x = F.dropout(x, p=m1_dropout)
        x = self.dense2(x)
        # x = F.log_softmax(x)  # this step is part of CrossEntropyLoss
        return x


model3 = Model3().to(device)
optimizer3 = optim.Adam(model3.parameters(), lr=learning_rate)

In [15]:
def load_eval_data(path):
    train_eval, valid_eval = np.loadtxt(path, delimiter=',')
    return train_eval, valid_eval

def plot_eval(train_eval, valid_eval):

    train_losses = train_eval[:, 0]
    train_accus  = train_eval[:, 1]
    valid_losses = valid_eval[:, 0]
    valid_accus  = valid_eval[:, 1]
    min_loss_idx = np.argmin(valid_losses)
    max_accu_idx = np.argmax(valid_accus)

    plt.figure(figsize=(12,4))
    plt.subplot(121)
    plt.plot(train_losses, label="train")
    plt.plot(valid_losses, label="validation")
    plt.scatter(min_loss_idx, valid_losses[min_loss_idx], c='r', label="minimum")
    plt.title("evaluation of losses")
    plt.xlabel("epochs")
    plt.legend()
    plt.subplot(122)
    plt.plot(train_accus, label="train")
    plt.plot(valid_accus, label="validation")
    plt.scatter(max_accu_idx, valid_accus[max_accu_idx], c='r', label="maximum")
    plt.title("evaluation of accuracy")
    plt.xlabel("epochs")
    plt.legend()
    plt.show()

def gen(model, sentence, iterations=10):

    sequences = tokenizer.texts_to_sequences([sentence])
    seq = [item for sublist in sequences for item in sublist] # tokenized words
    if(len(seq) < train_len): return -1
    
    for i in range(iterations):
        iput = np.array(seq[-19:])
        iput = np.vstack(iput).T
        iput = torch.from_numpy(iput).type(torch.cuda.LongTensor)

        # oput = F.log_softmax( model(iput) )  # prob dist
        oput = model(iput)
        
        oput = oput.cpu().detach().numpy()[0]
        # print(np.average(oput))
        oput = np.argmax(oput) # one-hot to long
        seq.append(oput)

    pred_sen = ''
    for tk in seq:
        pred_sen += reverse_word_map[tk] + ' '
    return pred_sen

In [16]:
model_name = 'Model1.pth'
model = torch.load('./models/' + model_name)


In [17]:
sen = "Calls on Member States, in the light of the growing risks which the European Union is running in terms of energy security"
gen(model, sen)

RuntimeError: CUDA error: device-side assert triggered